In [ ]:
# nuclio: start-code

In [80]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd

In [90]:
def handler(context):
    context.logger.info("Reading features from feature vector")
    stock_list = []
    service = fs.get_online_feature_service("stocks-vec")
    for key,value in context.sym_to_url.items():
        context.logger.info(key)
        context.logger.info(service.get([{"symbol": key}],as_list = True))
        #stock_list += service.get([{"symbol": key}],as_list = True)
    #df = pd.DataFrame(stock_list,index = [item[0] for item in context.sym_to_url.items()])
    #df.index.name = "symbol"
#     df.columns = ["price","volume","last_updated","sentiment"]
    #return df

In [94]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    mlrun.set_environment(project="stocks-test-avia")
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    setattr(context, 'news', "news")
    setattr(context, 'stocks', "stocks")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.*",
               "news.*"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [83]:
# nuclio: end-code

In [95]:
# test local
init_context(context)
df = handler(context)

Python> 2021-03-21 08:54:18,721 [info] Initalizing context & constructing featrure vector
Python> 2021-03-21 08:54:18,791 [info] Reading features from feature vector


,0,1,2,3,4,5,6,7,8,9,10
symbol,,,,,,,,,,,
GOOGL,2024.939941,2024.939941,4049.879883,61282.0,61282.0,2024.939941,61282.0,2021-03-19 15:59:00-04:00,-1.333333,-1.333333,-1.333333
MSFT,229.910004,229.910004,459.820007,656800.0,656800.0,229.910004,656800.0,2021-03-19 15:59:00-04:00,-1.666667,-1.666667,-1.666667
AMZN,3072.989990,3072.989990,6145.979980,115780.0,115780.0,3072.989990,115780.0,2021-03-19 15:59:00-04:00,-2.000000,-2.000000,-2.000000
AAPL,119.870003,119.870003,239.740005,2036818.0,2036818.0,119.870003,2036818.0,2021-03-19 15:59:00-04:00,3.000000,3.000000,3.000000


In [99]:
stock_feature_set = fs.FeatureSet("stocks_vec", entities=[fs.Entity("symbol")])

In [78]:
stock_feature_set.add_aggregation("sentiments","sentiment",["min","max"],["1h"],"10m")
stock_feature_set.add_aggregation("prices","price",["min","max","sum"],["1h"],"10m")
stock_feature_set.add_aggregation("volumes","volume",["min","max"],["1h"],"10m")

In [98]:
fs.infer_metadata(
    stock_feature_set,
    df,
    entity_columns=["symbol"],
    options=fs.InferOptions.default(),
)
[prices_min_1h,prices_max_1h,volumes_min_1h,volumes_max_1h,price,volume,last_updated,sentiments_min_1h]

TypeError: infer() missing 1 required positional argument: 'source'

In [105]:
stock_feature_set.to_dict()

{'kind': 'FeatureSet',
 'metadata': {'name': 'stocks_vec'},
 'spec': {'entities': [{'name': 'symbol', 'value_type': 'str'}],
  'partition_keys': [],
  'source': {'path': 'None'},
  'graph': {'engine': 'async'}},
 'status': {'state': 'created', 'preview': []}}